In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    max_tokens=200,
    
)

In [7]:
pdf_link = "ProjetoLei.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [8]:
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)

chunks = text_spliter.split_documents(pages)

In [ ]:
db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")

In [ ]:

vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

# Load Retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Chain - Contrução da cadeira de prompt para chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

In [16]:
def ask(question):
    context = retriever.invoke(question)  
    answer = chain.invoke({               
        "input_documents": context,
        "question": question
    })["output_text"]
    return answer, context

In [17]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

Answer:  Os principais pontos da Lei de Inteligência Artificial mencionada incluem:

1. Estabelecimento de direitos para a proteção das pessoas naturais afetadas por sistemas de inteligência artificial, incluindo acesso à informação e compreensão das decisões tomadas por esses sistemas.

2. Garantia do direito de contestar decisões automatizadas e solicitar intervenção humana.

3. Proteção contra discriminação e correção de vieses discriminatórios em sistemas de inteligência artificial.

4. Criação de uma regulação baseada em riscos, com uma modelagem regulatória fundada em direitos.

5. Estabelecimento de instrumentos de governança para garantir a transparência e prestação de contas dos agentes que desenvolvem e utilizam inteligência artificial.

6. Exigência de que os sistemas de IA destinados a grupos vulneráveis sejam desenvolvidos de forma que esses indivíduos consigam entender seu funcionamento e seus direitos.

7. Direito da pessoa afetada de solicitar explicações sobre decisões

In [18]:
for c in context:
    print(c)

page_content='liberdades fundamentais, a valorização do trabalho e da dignidade da pessoa 
humana e a inovação tecnológica representada pela inteligência artificial. 
O projeto tem um duplo objetivo. De um lado, estabelece direitos 
para proteção do elo mais vulnerável em questão, a pessoa natural que já é 
diariamente impactada por sistemas de inteligência artificial, desde a 
recomendação de conteúdo e direcionamento de publicidade na Internet até a 
sua análise de elegibilidade para tomada de crédito e para determinadas 
políticas públicas. De outro lado, ao dispor de ferramentas de governança e de 
um arranjo institucional de fiscalização e supervisão, cria condições de 
previsibilidade acerca da sua interpretação e, em última análise, segurança 
jurídica para inovação e o desenvolvimento tecnológico. 
A proposição parte da premissa, portanto, de que não há um trade-
off entre a proteção de direitos e liberdades fundamentais, da valorização do 
trabalho e da dignidade da pessoa hum